In [1]:
# Import dependencies
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select your features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=250,learning_rate=1.0,max_depth=1,random_state=0)
model.fit(X,y)
model.feature_importances_

array([2.60688858e-01, 2.46980546e-01, 1.75985959e-01, 2.03134802e-02,
       2.32487160e-03, 4.59675115e-04, 1.85726096e-04, 5.00055556e-03,
       4.10157937e-04, 2.72422069e-04, 4.31920796e-03, 9.41308828e-04,
       2.38202743e-03, 8.88944750e-03, 3.83105489e-02, 2.31265322e-04,
       2.50125549e-03, 1.67951790e-03, 4.52128956e-04, 2.29894003e-02,
       9.63857808e-04, 5.42085677e-03, 1.26294956e-04, 2.86544032e-04,
       7.19421013e-04, 4.61592239e-04, 7.96938647e-02, 2.74682994e-03,
       9.71028723e-04, 9.81578136e-02, 1.08029809e-03, 8.15077613e-04,
       3.65355791e-04, 1.71499278e-03, 1.93201650e-03, 1.31314682e-03,
       9.42795086e-04, 2.23334203e-03, 1.79633572e-03, 2.94017676e-03])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_nt        0.260689
koi_fpflag_ss        0.246981
koi_fpflag_co        0.175986
koi_steff_err1       0.098158
koi_model_snr        0.079694
koi_duration_err1    0.038311
koi_prad             0.022989
koi_fpflag_ec        0.020313
koi_duration         0.008889
koi_prad_err2        0.005421
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_steff_err1,koi_model_snr,koi_duration_err1,koi_prad,koi_fpflag_ec,koi_duration,koi_prad_err2
6947,1,0,0,50,11.5,0.83600,14.87,0,6.2780,-4.53
1614,0,1,0,78,16.9,0.10000,2.01,0,1.6390,-0.24
607,0,0,0,85,17.4,0.15800,2.22,0,3.4310,-0.52
4051,1,0,0,135,628.2,0.00403,37.47,0,1.7495,-5.59
1093,0,0,0,75,38.2,0.11600,2.88,0,6.4010,-0.68


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [11]:
from sklearn.svm import SVC

# Initialize model
model = SVC(kernel='poly')

# Train the model
model.fit(X_train_scaled, y_train)

SVC(kernel='poly')

In [12]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8308776425368354
Testing Data Score: 0.8227902946273831


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

# Create the GridSearchCV model
param_grid = [{'C':[1, 5, 10, 50,100], 'kernel':['poly']},
             {'C':[1, 5, 10, 50], 'kernel':['rbf'], 'gamma': [0.0001, 0.0005, 0.001, 0.005]}]

grid = GridSearchCV(model, param_grid, verbose=3)

In [14]:
# Train the new model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 21 candidates, totalling 105 fits
[CV 1/5] END ..................C=1, kernel=poly;, score=0.846 total time=   0.1s
[CV 2/5] END ..................C=1, kernel=poly;, score=0.816 total time=   0.1s
[CV 3/5] END ..................C=1, kernel=poly;, score=0.844 total time=   0.1s
[CV 4/5] END ..................C=1, kernel=poly;, score=0.812 total time=   0.1s
[CV 5/5] END ..................C=1, kernel=poly;, score=0.830 total time=   0.1s
[CV 1/5] END ..................C=5, kernel=poly;, score=0.874 total time=   0.1s
[CV 2/5] END ..................C=5, kernel=poly;, score=0.838 total time=   0.1s
[CV 3/5] END ..................C=5, kernel=poly;, score=0.863 total time=   0.1s
[CV 4/5] END ..................C=5, kernel=poly;, score=0.834 total time=   0.1s
[CV 5/5] END ..................C=5, kernel=poly;, score=0.857 total time=   0.1s
[CV 1/5] END .................C=10, kernel=poly;, score=0.886 total time=   0.1s
[CV 2/5] END .................C=10, kernel=poly

[CV 2/5] END .....C=50, gamma=0.005, kernel=rbf;, score=0.812 total time=   0.3s
[CV 3/5] END .....C=50, gamma=0.005, kernel=rbf;, score=0.848 total time=   0.3s
[CV 4/5] END .....C=50, gamma=0.005, kernel=rbf;, score=0.810 total time=   0.3s
[CV 5/5] END .....C=50, gamma=0.005, kernel=rbf;, score=0.824 total time=   0.3s


In [15]:
print(best_model.best_params_)
print("Best Score: ", best_model.best_score_)

{'C': 100, 'kernel': 'poly'}
Best Score:  0.8765714363900063


In [16]:
# Predict with best_model
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
4982,FALSE POSITIVE,FALSE POSITIVE
4866,CANDIDATE,CANDIDATE
2934,FALSE POSITIVE,FALSE POSITIVE
5007,FALSE POSITIVE,FALSE POSITIVE
3869,FALSE POSITIVE,FALSE POSITIVE


In [17]:
# Score the model
best_model.score(X_test_scaled, y_test)

0.8747833622183708

# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'SVCTwoKernelsNewFeatures.sav'
joblib.dump(best_model, filename)

['SVCTwoKernelsNewFeatures.sav']

In [19]:
# Print Classification Report.
from sklearn.metrics import classification_report
print(classification_report(y_test, grid_predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.80      0.67      0.73       555
     CONFIRMED       0.75      0.83      0.79       628
FALSE POSITIVE       0.98      1.00      0.99      1125

      accuracy                           0.87      2308
     macro avg       0.84      0.83      0.84      2308
  weighted avg       0.87      0.87      0.87      2308

